## Import des paquets Python

In [14]:
# Import des paquets requis et configuration du ROS Master
import os, sys
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))
from lab_utils.ip_config import get_ip
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/usr/lib/python3/dist-packages')
from os import environ
import rospy
from std_msgs.msg import Float32MultiArray
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose, PoseStamped, PoseWithCovarianceStamped
from sensor_msgs.msg import Imu
from sensor_msgs.msg import CompressedImage
from apriltag_ros.msg import AprilTagDetectionArray

In [15]:
VOTRE_NUMERO_EQUIPE = 1
SIM = True
ADMIN = False
VOTRE_NUMERO_ROBOT = 7

In [16]:
if SIM:
    environ['ROS_MASTER_URI'] = f"http://localhost:113{70+VOTRE_NUMERO_EQUIPE if not ADMIN else 11}/"
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-0{}.local:11311/".format(VOTRE_NUMERO_ROBOT)
    environ['ROS_IP'] = get_ip() # adresse IP de votre station de travail

In [17]:
rospy.init_node('dingo_controller', anonymous=True)
rate = rospy.Rate(50)

## Création des 'callbacks'
Chaque fonction est associée à un sujet ROS et enregistre en continue sont contenu dans une variable globale.

In [18]:
# Odometry subscriber callback
odom_msg = Pose()
def odom_callback(msg):
    global odom_msg
    odom_msg  = msg.pose.pose

In [19]:
# Realsense Pose subscriber callback
# pose_msg = Pose()
# def pose_callback(msg):
#     global pose_msg
#     pose_msg = msg.pose.pose
    # pose = PoseStamped()

    # pose.header.seq = 1
    # pose.header.stamp = rospy.Time.now()
    # pose.header.frame_id = "map"

    # pose.pose.position.x = pose_msg.position.x
    # pose.pose.position.y = pose_msg.position.y
    # pose.pose.position.z = 0.0

    # pose.pose.orientation.x = pose_msg.orientation.x
    # pose.pose.orientation.y = pose_msg.orientation.y
    # pose.pose.orientation.z = pose_msg.orientation.z
    # pose.pose.orientation.w = pose_msg.orientation.w

    # pose_pub.publish(pose)

In [20]:
# AMCL_pose callback (pour la visualisation de la pose)
pose_msg = Pose()
def amcl_callback(msg):
    global pose_msg
    pose_msg = msg.pose.pose
    pose = PoseStamped()

    pose.header.seq = 1
    pose.header.stamp = rospy.Time.now()
    pose.header.frame_id = "map"

    pose.pose.position.x = pose_msg.position.x
    pose.pose.position.y = pose_msg.position.y
    pose.pose.position.z = 0.0

    pose.pose.orientation.x = pose_msg.orientation.x
    pose.pose.orientation.y = pose_msg.orientation.y
    pose.pose.orientation.z = pose_msg.orientation.z
    pose.pose.orientation.w = pose_msg.orientation.w

    pose_pub.publish(pose)

In [21]:
# AprilTag subscriber callback
tag_array = AprilTagDetectionArray()
def tag_callback(msg):
    global tag_array
    tag_array  = msg

In [22]:
# Imu subscriber callback
imu_msg = Imu()
def imu_callback(msg):
    global imu_msg
    imu_msg = msg

In [23]:
# Camera subscriber callback
cam_msg = CompressedImage()
def cam_callback(msg):
    global cam_msg
    cam_msg = msg

In [24]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
pose_pub = rospy.Publisher('/mobile_manip/pose', PoseStamped, queue_size=1)
amcl_sub = rospy.Subscriber('/amcl_pose', PoseWithCovarianceStamped, amcl_callback)
#pose_sub = rospy.Subscriber('/mobile_manip/dingo_velocity_controller/odom', Odometry, odom_callback)
#pose_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, pose_callback)
imu_sub = rospy.Subscriber('/imu/data', Imu, imu_callback)
cam_sub = rospy.Subscriber('/mobile_manip/d435i/color/image_raw/compressed', CompressedImage, cam_callback)
tag_sub = rospy.Subscriber('/tag_detections', AprilTagDetectionArray, tag_callback)

## Contrôleur du mouvement

In [25]:
interwheel_distance = 0.46
left_wheel_radius = 0.045
right_wheel_radius = 0.045
def move_robot(linear, angular):
    vel_left  = (linear - angular * interwheel_distance / 2.0) / left_wheel_radius
    vel_right = (linear + angular * interwheel_distance / 2.0) / right_wheel_radius

    # Envoi des commandes au roues par topic ROS
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = vel_left
    cmd_drive_msg.drivers[1] = vel_right
    cmd_drive_pub.publish(cmd_drive_msg)

## Interface manuelle
Permet le contrôle du robot avec des flèches et un retour visuel de la caméra frontale.

In [26]:
from ipywidgets import HBox, Box, Layout, GridspecLayout
import ipywidgets as widgets
import threading
import time
# Image update done with threading, maybe look at ipywebrtc for upgrade

# JUPYTER-ROS
# v = ros3d.Viewer()
# rc = ros3d.ROSConnection(url="ws://localhost:9090")
# tf_client = ros3d.TFClient(ros=rc, fixed_frame='/map')
# g = ros3d.GridModel()
# laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
# map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
# pose_view = ros3d.Pose(topic="/mobile_manip/pose", ros=rc, tf_client=tf_client)
# v.objects = [laser_view, map_view, pose_view]
# v.layout = Layout(border="3px solid black", width="700px", height="600px")

image = widgets.Image(
        value=cam_msg.data,
        format="png",
        height="120%",
        width="200%")

btn_up = widgets.Button(icon='arrow-up')
btn_left = widgets.Button(icon='arrow-left')
btn_down = widgets.Button(icon='arrow-down')
btn_right = widgets.Button(icon='arrow-right')
btn_stop = widgets.Button(description='Stop')

def update_plot():
    while True:
        image.value=cam_msg.data
        time.sleep(0.1)

def on_btn_up_clicked(b):
    move_robot(0.5,0)
        
def on_btn_left_clicked(b):
    move_robot(0,0.5)
        
def on_btn_down_clicked(b):
    move_robot(-0.5,0)
        
def on_btn_right_clicked(b):
    move_robot(0,-0.5)
    
def on_btn_stop_clicked(b):
    move_robot(0,0)

btn_up.on_click(on_btn_up_clicked)
btn_left.on_click(on_btn_left_clicked)
btn_down.on_click(on_btn_down_clicked)
btn_right.on_click(on_btn_right_clicked)
btn_stop.on_click(on_btn_stop_clicked)

# JUPYTER-ROS
# form = Box(image, layout=Layout(
#     display='flex',
#     flex_flow='column',
#     border='solid 2px',
#     align_items='stretch',
#     width='50%',
#     padding='10px'
# ))

# box = HBox(children=[image, v, btn_up, btn_stop])
# box.layout.padding = "0px"
# box

grid = GridspecLayout(8, 8,height='600px')

grid[0:4,0:3] = image
grid[5,1] = btn_up
grid[6,0] = btn_left
grid[6,1] = btn_stop
grid[6,2] = btn_right
grid[7,1] = btn_down
#grid[:,4:7] = v
thread = threading.Thread(target=update_plot)
thread.start()
grid

GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xf…